In [1]:
import numpy as np
import pandas as pd
import os
import pywt

from scipy.stats import zscore
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.metrics import f1_score


In [2]:
windowLength = 4 # sample window length, in secs
stride = 0.1  # moving window stride, in secs
overlap = windowLength - stride
ogFs = 5000 # original sampling rate, in Hz
dsFs = 200 # sampling rate, in Hz
nSamples = int(dsFs * windowLength)
nStride = int(dsFs * stride)
nOverlap = int(dsFs * overlap)
fft = True
wavelet = False
if fft:
    nFft = 64 # FFT bin size
else:
    nFft = 1

In [3]:
# Create a label encoder
le = LabelEncoder()

# Specify the directory path
directory = '/Users/zhewei/Library/CloudStorage/GoogleDrive-zhewei@umich.edu/My Drive/BYB_EEG_Classfication/Data_Trimmed_200Hz_20s/bundled'

# Read all CSV files in the directory
dataframes = []
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        # Fit the encoder to the string column and  transform it to numerical data
        df['EyesLabel'] = le.fit_transform(df['EyesLabel'])
        # Normalize the EEG data
        df['EEGdata']= zscore(df['EEGdata'])
        dataframes.append(df)

        # Concatenate all dataframes into a single dataframe
        combined_df = pd.concat(dataframes)

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dataframes)

# extract the spectral features from the EEG data
# fft 64 points
feature_df = pd.DataFrame()

for i in range(0, len(combined_df), nSamples - nOverlap):
    # Get the current window
    windowData = combined_df['EEGdata'].iloc[i:i + nSamples]
    wavelet_transform = pywt.wavedec(windowData, 'haar', level=2)

    # Check if the window is of the correct length
    if len(windowData) < nSamples:
        break
    # Compute the spectrogram
    else:
        fft_result = np.fft.fft(windowData, n=nFft)
        fft_result = np.abs(fft_result) 
        # now that every window has 64 spectral features
        # we want to give each of them a column name
        feature_df.loc[i, 'EyesLabel'] = combined_df['EyesLabel'].iloc[i]
        for j in range(nFft):
            feature_df.loc[i, f'fft_{j}'] = fft_result[j]

# zero pad the feature_df
feature_df = feature_df.fillna(0)

In [4]:
# Split the data into train and test sets
train_df, test_df = train_test_split(feature_df, test_size=0.2, random_state=42)

# Split the train set into train and validation sets
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Print the shapes of the train, test, and validation sets
print("Train set shape:", train_df.shape)
print("Test set shape:", test_df.shape)
print("Validation set shape:", val_df.shape)

Train set shape: (9958, 65)
Test set shape: (3113, 65)
Validation set shape: (2490, 65)


In [9]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Number of splits
n_splits = 5

# Initialize TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=n_splits)

# Assume 'feature_df' is your entire dataset
# Reserve last 20% of data as the test set
test_size = int(len(feature_df) * 0.2)
train_val_df = feature_df[:-test_size]
test_df = feature_df[-test_size:]

# Use TimeSeriesSplit to create train/validation sets
for train_index, val_index in tscv.split(train_val_df):
    train_df = train_val_df.iloc[train_index]
    val_df = train_val_df.iloc[val_index]
    print(f"Train set shape: {train_df.shape}")
    print(f"Validation set shape: {val_df.shape}")

# Now print the test set shape
print("Test set shape:", test_df.shape)


Train set shape: (2079, 65)
Validation set shape: (2074, 65)
Train set shape: (4153, 65)
Validation set shape: (2074, 65)
Train set shape: (6227, 65)
Validation set shape: (2074, 65)
Train set shape: (8301, 65)
Validation set shape: (2074, 65)
Train set shape: (10375, 65)
Validation set shape: (2074, 65)
Test set shape: (3112, 65)


In [10]:
# Separate features and labels
train_features = torch.tensor(train_df.iloc[:, 1:].values.astype(np.float32))  # all columns except the first one
train_labels = torch.tensor(train_df.iloc[:, 0].values.astype(np.float32))  # only the first column
train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=False, num_workers=4,persistent_workers=True)


#  Validate Set
val_features = torch.tensor(val_df.iloc[:, 1:].values.astype(np.float32))
val_labels = torch.tensor(val_df.iloc[:, 0].values.astype(np.float32))
val_dataset = torch.utils.data.TensorDataset(val_features, val_labels)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4,persistent_workers=True)


# test
test_features = torch.tensor(test_df.iloc[:, 1:].values.astype(np.float32))  # all columns except the first one
test_labels = torch.tensor(test_df.iloc[:, 0].values.astype(np.float32))  # only the first column
test_dataset = torch.utils.data.TensorDataset(test_features, test_labels)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4,persistent_workers=True)

In [17]:
from torch.utils.data import DataLoader
from torch import optim
import torchmetrics
from pytorch_lightning.callbacks import EarlyStopping

# Step 1: Define the GRU network architecture
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,dropout=0.5):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output, hidden = self.gru(x)
        last_hidden_state = hidden[-1]  
        output = self.fc(last_hidden_state)
        output = self.sigmoid(output)
        return output

# Define the input size, hidden size, and output size
input_size = nFft  # Assuming one feature (EEGData) as input
hidden_size = 50  # Number of hidden units in the GRU layer
batch_size = 32  # Number of samples in each batch
output_size = batch_size  # Number of output units (labels)

# Create an instance of the GRU network
gru_net = GRUNetwork(input_size, hidden_size, output_size)

# Print the network architecture
print(gru_net)

# Step 2: Prepare the data
# Create PyTorch DataLoader objects for each set

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4,persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4,persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4,persistent_workers=True)

# Step 3: Define the training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
num_epochs = 10

model = GRUNetwork(input_size, hidden_size, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)

# Metrics
accuracy = torchmetrics.Accuracy(task='binary',threshold=0.5).to(device)
f1_score = torchmetrics.F1Score(task='binary',threshold=0.5).to(device)


for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0

    for inputs,labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        if len(labels) != batch_size or len(inputs) != batch_size:
            continue

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        
        # Update metrics
        accuracy.update(outputs, labels.float())
        f1_score.update(outputs, labels.float())

    # Log epoch metrics
    train_acc = accuracy.compute()
    train_f1 = f1_score.compute()
    print(f"Epoch {epoch+1}: Accuracy: {train_acc}, F1 Score: {train_f1}")

    # Reset metrics for next epoch
    accuracy.reset()
    f1_score.reset()

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_accuracy = 0.0

    class EarlyStopping:
        def __init__(self, patience=7, min_delta=0):
            self.patience = patience
            self.min_delta = min_delta
            self.counter = 0
            self.best_score = None
            self.early_stop = False

        def __call__(self, val_acc, model):
            score = val_acc

            if self.best_score is None:
                self.best_score = score
            elif score < self.best_score + self.min_delta:
                self.counter += 1
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.counter = 0

    # Initialize early stopping
    early_stopping = EarlyStopping(patience=10, min_delta=0.01)


    with torch.no_grad():
        for inputs, labels in val_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)
            if len(labels) != batch_size or len(inputs) != batch_size:
                continue

            outputs = model(inputs)
            loss = criterion(outputs, labels.float())

            # Update validation metrics
            accuracy.update(outputs, labels.float())
            f1_score.update(outputs, labels.float())

        # Log validation metrics
        val_acc = accuracy.compute()
        val_f1 = f1_score.compute()
        # implement the early stopping callback
         
        print(f"Epoch {epoch+1}: Val Accuracy: {val_acc}, Val F1 Score: {val_f1}")


        accuracy.reset()
        f1_score.reset()

        # Call early stopping
        early_stopping(val_acc, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

    # Implement early stopping or save the best model based on validation accuracy here

# Evaluation loop
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        if len(labels) != batch_size or len(inputs) != batch_size:
            continue
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        # Update metrics
        accuracy.update(outputs, labels.float())
        f1_score.update(outputs, labels.float())

test_acc = accuracy.compute()
test_f1 = f1_score.compute()
print(f"Test Accuracy: {test_acc}, Test F1 Score: {test_f1}")
 

GRUNetwork(
  (gru): GRU(64, 50, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=50, out_features=32, bias=True)
  (sigmoid): Sigmoid()
)
Epoch 1: Accuracy: 0.5407986044883728, F1 Score: 0.5727362632751465
Epoch 1: Val Accuracy: 0.58349609375, Val F1 Score: 0.6207203269004822
Epoch 2: Accuracy: 0.6184413433074951, F1 Score: 0.6427023410797119
Epoch 2: Val Accuracy: 0.64208984375, Val F1 Score: 0.6639156341552734
Epoch 3: Accuracy: 0.6656057238578796, F1 Score: 0.6791300177574158
Epoch 3: Val Accuracy: 0.66748046875, Val F1 Score: 0.615905225276947
Epoch 4: Accuracy: 0.6923225522041321, F1 Score: 0.6980310678482056
Epoch 4: Val Accuracy: 0.70703125, Val F1 Score: 0.7047244310379028
Epoch 5: Accuracy: 0.7227044701576233, F1 Score: 0.7331786751747131
Epoch 5: Val Accuracy: 0.671875, Val F1 Score: 0.718120813369751
Epoch 6: Accuracy: 0.7594521641731262, F1 Score: 0.7722790241241455
Epoch 6: Val Accuracy: 0.650390625, Val F1 Score: 0.5803048014640808


In [ ]:
from torch.utils.data import DataLoader
from torch import optim
import torchmetrics
from pytorch_lightning.callbacks import EarlyStopping

# Step 1: Define the GRU network architecture
class GRUNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size,dropout=0.5):
        super(GRUNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output, hidden = self.gru(x)
        last_hidden_state = hidden[-1]  
        output = self.fc(last_hidden_state)
        output = self.sigmoid(output)
        return output

# Define the input size, hidden size, and output size
input_size = nFft  # Assuming one feature (EEGData) as input
hidden_size = 50  # Number of hidden units in the GRU layer
batch_size = 32  # Number of samples in each batch
output_size = batch_size  # Number of output units (labels)

# Create an instance of the GRU network
gru_net = GRUNetwork(input_size, hidden_size, output_size)

# Print the network architecture
print(gru_net)

# Step 2: Prepare the data
# Create PyTorch DataLoader objects for each set

# Create a DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4,persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4,persistent_workers=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4,persistent_workers=True)

# Step 3: Define the training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
learning_rate = 0.001
num_epochs = 10

model = GRUNetwork(input_size, hidden_size, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)

# Metrics
accuracy = torchmetrics.Accuracy(task='binary',threshold=0.5).to(device)
f1_score = torchmetrics.F1Score(task='binary',threshold=0.5).to(device)


for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_accuracy = 0.0

    for inputs,labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        if len(labels) != batch_size or len(inputs) != batch_size:
            continue

        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        
        # Update metrics
        accuracy.update(outputs, labels.float())
        f1_score.update(outputs, labels.float())

    # Log epoch metrics
    train_acc = accuracy.compute()
    train_f1 = f1_score.compute()
    print(f"Epoch {epoch+1}: Accuracy: {train_acc}, F1 Score: {train_f1}")

    # Reset metrics for next epoch
    accuracy.reset()
    f1_score.reset()

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_accuracy = 0.0

    # Define the early stopping callback
    early_stop_callback = EarlyStopping(
        monitor='val_acc',
        min_delta=0.00,
        patience=10,
        verbose=False,
        mode='max'  # Change to 'max' because we want to maximize accuracy
        )

    with torch.no_grad():
        for inputs, labels in val_loader:

            inputs = inputs.to(device)
            labels = labels.to(device)
            if len(labels) != batch_size or len(inputs) != batch_size:
                continue

            outputs = model(inputs)
            loss = criterion(outputs, labels.float())

            # Update validation metrics
            accuracy.update(outputs, labels.float())
            f1_score.update(outputs, labels.float())

        # Log validation metrics
        val_acc = accuracy.compute()
        val_f1 = f1_score.compute()
        # implement the early stopping callback
        early_stop_callback.on_validation_end(pl_module=model, val_acc)

        
        print(f"Epoch {epoch+1}: Val Accuracy: {val_acc}, Val F1 Score: {val_f1}")


        accuracy.reset()
        f1_score.reset()

    # Implement early stopping or save the best model based on validation accuracy here

# Evaluation loop
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        if len(labels) != batch_size or len(inputs) != batch_size:
            continue
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        # Update metrics
        accuracy.update(outputs, labels.float())
        f1_score.update(outputs, labels.float())

test_acc = accuracy.compute()
test_f1 = f1_score.compute()
print(f"Test Accuracy: {test_acc}, Test F1 Score: {test_f1}")
 

# visualize the above GRU model

In [19]:
# visualize the Gated Recurrent Unit (GRU) network architecture
from torchsummary import summary
summary(model, input_size=(batch_size, nFft), device=device.type)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 32, 50], [-1, 2, 50]]               0
            Linear-2                   [-1, 32]           1,632
           Sigmoid-3                   [-1, 32]               0
Total params: 1,632
Trainable params: 1,632
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.22
Params size (MB): 0.01
Estimated Total Size (MB): 1.23
----------------------------------------------------------------
